In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.5
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
views_programmes_schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("ts_start", IntegerType()),
    StructField("ts_end", IntegerType()),
    StructField("item_type", StringType())
])


In [3]:
%%time
dfProgrammes = spark.read\
        .format("csv")\
        .load("/labs/laba03/lab10_views_programmes.csv", header='true', schema = views_programmes_schema)\
        .coalesce(3)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 972 ms


In [4]:
dfProgrammes.take(12)

[Row(user_id=0, item_id=7101053, ts_start=1491409931, ts_end=1491411600, item_type='live'),
 Row(user_id=0, item_id=7101054, ts_start=1491412481, ts_end=1491451571, item_type='live'),
 Row(user_id=0, item_id=7101054, ts_start=1491411640, ts_end=1491412481, item_type='live'),
 Row(user_id=0, item_id=6184414, ts_start=1486191290, ts_end=1486191640, item_type='live'),
 Row(user_id=257, item_id=4436877, ts_start=1490628499, ts_end=1490630256, item_type='live'),
 Row(user_id=1654, item_id=7489015, ts_start=1493434801, ts_end=1493435401, item_type='live'),
 Row(user_id=1654, item_id=7489023, ts_start=1493444101, ts_end=1493445601, item_type='live'),
 Row(user_id=1654, item_id=6617053, ts_start=1489186156, ts_end=1489200834, item_type='live'),
 Row(user_id=1654, item_id=6438693, ts_start=1487840070, ts_end=1487840433, item_type='live'),
 Row(user_id=1654, item_id=6526859, ts_start=1488705452, ts_end=1488706154, item_type='live'),
 Row(user_id=1654, item_id=6526754, ts_start=1488532396, ts_end

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, IntegerType
views_train_schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("purchase", IntegerType())
])

In [6]:
%%time
dfTrain = spark.read\
        .format("csv")\
        .load("/labs/laba03/lab10_train.csv", header='true', schema=views_train_schema)\
        .coalesce(3)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 24.7 ms


In [7]:
dfTrain.filter((dfTrain.user_id == 1654) & (dfTrain.purchase == 0)).show(5)
dfTrain.filter((dfTrain.user_id == 1654) & (dfTrain.purchase == 1)).show(5)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
|   1654|  89901|       0|
|   1654| 100504|       0|
+-------+-------+--------+
only showing top 5 rows

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|   9897|       1|
|   1654|   7394|       1|
|   1654|   9064|       1|
|   1654|  73216|       1|
|   1654|  88816|       1|
+-------+-------+--------+



In [8]:
%%time
dfTest = spark.read\
        .format("csv")\
        .load("/labs/laba03/lab10_test.csv", header='true', schema=views_train_schema)\
        .coalesce(3)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 14.1 ms


In [9]:
dfTest.show(10)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  94814|    null|
|   1654|  93629|    null|
|   1654|   9980|    null|
|   1654|  95099|    null|
|   1654|  11265|    null|
|   1654|  88896|    null|
|   1654|  67740|    null|
|   1654|  74271|    null|
|   1654|  99871|    null|
|   1654|  78570|    null|
+-------+-------+--------+
only showing top 10 rows



In [10]:
%%time
dfItems = spark.read\
          .format("csv")\
          .load("/labs/laba03/lab10_items.csv", header='true', sep='\t', inferShema='true')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 501 ms


In [11]:
dfItems.printSchema() #???

root
 |-- item_id: string (nullable = true)
 |-- channel_id: string (nullable = true)
 |-- datetime_availability_start: string (nullable = true)
 |-- datetime_availability_stop: string (nullable = true)
 |-- datetime_show_start: string (nullable = true)
 |-- datetime_show_stop: string (nullable = true)
 |-- content_type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- region_id: string (nullable = true)



In [12]:
dfItems.take(3)

[Row(item_id='65667', channel_id=None, datetime_availability_start='1970-01-01T00:00:00Z', datetime_availability_stop='2018-01-01T00:00:00Z', datetime_show_start=None, datetime_show_stop=None, content_type='1', title='на пробах только девушки (all girl auditions)', year='2013.0', genres='Эротика', region_id=None),
 Row(item_id='65669', channel_id=None, datetime_availability_start='1970-01-01T00:00:00Z', datetime_availability_stop='2018-01-01T00:00:00Z', datetime_show_start=None, datetime_show_stop=None, content_type='1', title='скуби ду: эротическая пародия (scooby doo: a xxx parody)', year='2011.0', genres='Эротика', region_id=None),
 Row(item_id='65668', channel_id=None, datetime_availability_start='1970-01-01T00:00:00Z', datetime_availability_stop='2018-01-01T00:00:00Z', datetime_show_start=None, datetime_show_stop=None, content_type='1', title='горячие девочки для горячих девочек (hot babes 4 hot babes)', year='2011.0', genres='Эротика', region_id=None)]

In [13]:
from pyspark.sql import functions as f
from pyspark.sql import Window
from pyspark.sql.functions import col

In [14]:
df_genre_unpivot = dfTrain.join(dfItems, on="item_id")\
    .select(dfTrain.item_id, dfTrain.user_id, dfTrain.purchase, f.split(dfItems.genres,','))\
    .withColumn("genre",f.explode("split(genres, ,)"))\
    .drop("split(genres, ,)")\
    .coalesce(3)

In [15]:
df_genre_unpivot.show(10)

+-------+-------+--------+--------------------+
|item_id|user_id|purchase|               genre|
+-------+-------+--------+--------------------+
|  74107|   1654|       0|               Драмы|
|  74107|   1654|       0|          Зарубежные|
|  89249|   1654|       0|    Короткометражные|
|  89249|   1654|       0|          Зарубежные|
|  99982|   1654|       0|        Про животных|
|  99982|   1654|       0|Западные мультфильмы|
|  99982|   1654|       0|      Для всей семьи|
|  99982|   1654|       0|           Для детей|
|  99982|   1654|       0|          Зарубежные|
|  89901|   1654|       0|               Драмы|
+-------+-------+--------+--------------------+
only showing top 10 rows



In [16]:
df_genre_unpivot_test = dfTest.join(dfItems, on="item_id")\
    .select(dfTest.item_id, dfTest.user_id, dfTest.purchase, f.split(dfItems.genres,','))\
    .withColumn("genre",f.explode("split(genres, ,)"))\
    .drop("split(genres, ,)")\
    .coalesce(3)

In [17]:
df_genre_purchase =  df_genre_unpivot.filter("purchase == 1").coalesce(3).cache()

In [18]:
df_genre_purchase.show(10)

+-------+-------+--------+-------------------+
|item_id|user_id|purchase|              genre|
+-------+-------+--------+-------------------+
|   7394|   1654|       1|        Мультфильмы|
|   7394|   1654|       1|     Союзмультфильм|
|   7394|   1654|       1|               Наши|
|  73216|   1654|       1|               Наши|
|  73216|   1654|       1|Для самых маленьких|
|  88816|   1654|       1|        Приключения|
|  88816|   1654|       1|     Документальные|
|  88816|   1654|       1|         Зарубежные|
|  72067| 510087|       1|          Детективы|
|  72067| 510087|       1|              Драмы|
+-------+-------+--------+-------------------+
only showing top 10 rows



In [19]:
df_genre_train = df_genre_unpivot.alias("t1").join(df_genre_purchase.alias("t2"), on = f.expr("t1.user_id == t2.user_id and t1.genre == t2.genre"))\
    .filter(f.expr("t1.item_id <> t2.item_id"))\
    .groupby("t1.user_id", "t1.item_id")\
    .sum("t2.purchase")\
    .withColumnRenamed("sum(purchase)","userGenrePurchase")\
    .coalesce(3)

In [20]:
df_genre_test = df_genre_unpivot_test.alias("t1").join(df_genre_purchase.alias("t2"), on = f.expr("t1.user_id == t2.user_id and t1.genre == t2.genre"))\
    .filter(f.expr("t1.item_id <> t2.item_id"))\
    .groupby("t1.user_id", "t1.item_id")\
    .sum("t2.purchase")\
    .withColumnRenamed("sum(purchase)","userGenrePurchase")\
    .coalesce(3)

In [21]:
dfTrain = dfTrain\
    .withColumn("userPurchases", f.sum("purchase").over(Window.partitionBy("user_id")) - col("purchase"))\
    .withColumn("itemPopular", f.sum("purchase").over(Window.partitionBy("item_id")) - col("purchase"))\
    .join(dfProgrammes.filter("item_type == 'live'").groupby("user_id").count(), on="user_id",how="left")\
    .withColumn("live_watch_count", col("count") - col("purchase"))\
    .drop("count")\
    .join(dfProgrammes.filter("item_type == 'pvr'").groupby("user_id").count(), on="user_id",how="left")\
    .withColumn("pvr_watch_count", col("count") - col("purchase"))\
    .drop("count")\
    .join(dfItems.select("item_id", col("year").cast("double")), on="item_id",how="inner")\
    .join(df_genre_train, on=["item_id", "user_id"],how="left")\
    .fillna(0)\
    .coalesce(3).cache()

In [22]:
dfTrain.show(10)

+-------+-------+--------+-------------+-----------+----------------+---------------+------+-----------------+
|item_id|user_id|purchase|userPurchases|itemPopular|live_watch_count|pvr_watch_count|  year|userGenrePurchase|
+-------+-------+--------+-------------+-----------+----------------+---------------+------+-----------------+
|    326| 659698|       0|            1|          1|             126|            141|2012.0|                0|
|    326| 747141|       0|            1|          1|              87|            273|2012.0|                3|
|    326| 840492|       0|            0|          1|             222|            183|2012.0|                0|
|    326| 891603|       0|            5|          1|             266|            299|2012.0|                0|
|    326| 903828|       0|            1|          1|               1|              1|2012.0|                0|
|    326| 921813|       0|            1|          1|              23|              9|2012.0|                0|
|

In [23]:
dfTest = dfTest\
    .join(dfTrain.groupby("user_id").sum("purchase"), on="user_id",how="left")\
    .withColumnRenamed("sum(purchase)", "userPurchases")\
    .join(dfTrain.groupby("item_id").sum("purchase"), on="item_id",how="left")\
    .withColumnRenamed("sum(purchase)", "itemPopular")\
    .join(dfProgrammes.filter("item_type == 'live'").groupby("user_id").count(), on="user_id",how="left")\
    .withColumnRenamed("count", "live_watch_count")\
    .join(dfProgrammes.filter("item_type == 'pvr'").groupby("user_id").count(), on="user_id",how="left")\
    .withColumnRenamed("count", "pvr_watch_count")\
    .join(dfItems.select("item_id", col("year").cast("double")), on="item_id",how="inner")\
    .join(df_genre_test, on=["item_id", "user_id"],how="left")\
    .fillna(0)\
    .coalesce(3).cache()

In [24]:
dfTest.show(10)

+-------+-------+--------+-------------+-----------+----------------+---------------+------+-----------------+
|item_id|user_id|purchase|userPurchases|itemPopular|live_watch_count|pvr_watch_count|  year|userGenrePurchase|
+-------+-------+--------+-------------+-----------+----------------+---------------+------+-----------------+
|    326| 762457|       0|           69|          1|             513|             25|2012.0|               29|
|    326| 878144|       0|            1|          1|             386|              5|2012.0|                0|
|    326| 904529|       0|            0|          1|             110|             14|2012.0|                0|
|    326| 923368|       0|            0|          1|              55|              0|2012.0|                0|
|    336| 894930|       0|            1|          0|             283|             44|2012.0|                1|
|    357| 885902|       0|            2|          1|             371|             59|2012.0|                0|
|

In [25]:
from pyspark.ml.feature import VectorAssembler
assemblerInputs = ['userPurchases', 'itemPopular', 'live_watch_count', 'pvr_watch_count', 'year', 'userGenrePurchase']
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

df_features = assembler.transform(dfTrain).coalesce(3)
df_features_test = assembler.transform(dfTest).coalesce(3)

In [26]:
train = df_features.sampleBy("purchase", fractions={0: 0.75, 1: 0.75}).coalesce(3).cache()
test = df_features.join(train, (df_features.user_id == train.user_id) & (df_features.item_id == train.item_id)\
                        , how="leftanti").coalesce(3).cache()

In [27]:
train.show(5)
test.show(5)

+-------+-------+--------+-------------+-----------+----------------+---------------+------+-----------------+--------------------+
|item_id|user_id|purchase|userPurchases|itemPopular|live_watch_count|pvr_watch_count|  year|userGenrePurchase|            features|
+-------+-------+--------+-------------+-----------+----------------+---------------+------+-----------------+--------------------+
|    326| 659698|       0|            1|          1|             126|            141|2012.0|                0|[1.0,1.0,126.0,14...|
|    326| 747141|       0|            1|          1|              87|            273|2012.0|                3|[1.0,1.0,87.0,273...|
|    326| 840492|       0|            0|          1|             222|            183|2012.0|                0|[0.0,1.0,222.0,18...|
|    326| 933055|       0|            0|          1|              46|              0|2012.0|                0|[0.0,1.0,46.0,0.0...|
|    336| 847580|       0|            9|          0|              46|       

In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol='purchase')

In [29]:
%%time
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(featuresCol = 'features', labelCol = 'purchase', maxDepth=8, maxIter=15, seed = 42)
model = gbt.fit(train)

CPU times: user 28 ms, sys: 4 ms, total: 32 ms
Wall time: 2min 3s


In [30]:
predictions = model.transform(test)
predictions.select("user_id", "item_id", "purchase", "rawPrediction", "probability", "prediction").show(10)
predictions.filter(predictions.prediction == 1).select("user_id", "item_id", "purchase", "rawPrediction", "probability", "prediction").show(10)

+-------+-------+--------+--------------------+--------------------+----------+
|user_id|item_id|purchase|       rawPrediction|         probability|prediction|
+-------+-------+--------+--------------------+--------------------+----------+
|   1654|  67318|       0|[1.44554720307470...|[0.94740444172881...|       0.0|
|   1654|  73409|       0|[1.44521513863541...|[0.94737133883980...|       0.0|
|   1654|  77673|       0|[1.44554720307470...|[0.94740444172881...|       0.0|
|   1654|  95599|       0|[1.44554720307470...|[0.94740444172881...|       0.0|
| 510087|  79856|       0|[1.43340835550160...|[0.94618147831327...|       0.0|
| 510087|  92538|       0|[1.42814080026813...|[0.94564247987134...|       0.0|
| 510087|  93562|       0|[1.44244594234322...|[0.94709451557742...|       0.0|
| 510087|  96398|       0|[1.44439912856999...|[0.94728990869985...|       0.0|
| 510087|  97941|       0|[1.44510650417007...|[0.94736050500012...|       0.0|
| 517612|  72029|       0|[1.44565033526

In [31]:
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))


Test Area Under ROC: 0.935562156697861


In [32]:
%%time
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(featuresCol = 'features', labelCol = 'purchase', maxDepth=8, maxIter=15, seed = 42)
model = gbt.fit(df_features)

CPU times: user 32 ms, sys: 8 ms, total: 40 ms
Wall time: 3min 41s


In [33]:
predictions = model.transform(df_features_test)
predictions.select("user_id", "item_id", "rawPrediction", "probability", "prediction").take(20)
predictions.filter(predictions.prediction == 1).select("user_id", "item_id", "rawPrediction", "probability", "prediction").take(10)

[Row(user_id=717302, item_id=77442, rawPrediction=DenseVector([-0.2139, 0.2139]), probability=DenseVector([0.3947, 0.6053]), prediction=1.0),
 Row(user_id=588378, item_id=89624, rawPrediction=DenseVector([-1.0126, 1.0126]), probability=DenseVector([0.1166, 0.8834]), prediction=1.0),
 Row(user_id=588378, item_id=74390, rawPrediction=DenseVector([-0.3813, 0.3813]), probability=DenseVector([0.3181, 0.6819]), prediction=1.0),
 Row(user_id=833685, item_id=89624, rawPrediction=DenseVector([-0.3437, 0.3437]), probability=DenseVector([0.3346, 0.6654]), prediction=1.0),
 Row(user_id=878599, item_id=5510, rawPrediction=DenseVector([-0.4166, 0.4166]), probability=DenseVector([0.3029, 0.6971]), prediction=1.0),
 Row(user_id=834405, item_id=8658, rawPrediction=DenseVector([-0.1573, 0.1573]), probability=DenseVector([0.422, 0.578]), prediction=1.0),
 Row(user_id=851745, item_id=10467, rawPrediction=DenseVector([-0.134, 0.134]), probability=DenseVector([0.4334, 0.5666]), prediction=1.0),
 Row(user_id

In [34]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
import pyspark.sql.functions as f
secondElement=f.udf(lambda v:float(v[1]),FloatType())


In [35]:
result_df = predictions\
    .withColumn("purchase", secondElement("probability"))\
    .select("user_id", "item_id", "purchase")\
    .orderBy("user_id", "item_id")\
    .toPandas().to_csv('lab03.csv')

In [ ]:
sc.stop()